In [1]:
# Import our module containing helper functions
import gravann

# Core imports
import numpy as np
import scipy
import pickle as pk
import os
from collections import deque
import torchquad as tquad
tquad.set_log_level("CRITICAL")
from copy import deepcopy

# pytorch
from torch import nn
import torch

# plotting stuff
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib notebook

# Ensure that changes in imported module (gravann most importantly) are autoreloaded
%load_ext autoreload
%autoreload 2

# If possible enable CUDA
gravann.enableCUDA()
gravann.fixRandomSeeds()
device = os.environ["TORCH_DEVICE"]
print("Will use device ",device)

C:\ProgramData\Miniconda3\envs\geodesynet\lib\site-packages\pyvista\themes.py:122: PyvistaDeprecationWarning: use "dark" instead of "night" theme
  warnings.warn('use "dark" instead of "night" theme', PyvistaDeprecationWarning)
09:04:39|TQ-INFO| Initializing torchquad.


Available devices  1
__pyTorch VERSION: 1.11.0
__CUDNN VERSION: 8200
__Number CUDA Devices: 1
Active CUDA Device: GPU 0
Setting default tensor type to Float32
Will use device  cuda:0


In [2]:
# Load the models to compare, on of 
# "eros", ""

In [3]:
name = "eros"

In [4]:
# 1 - We load the masconCUBE
with open("models/"+name+"_masconCUBE.pk", "rb") as file:
    mascon_cube_points, mascon_cube_masses, mascon_cube_name = pk.load(file)
# 2 - We load the ground truth (also a mascon body)
with open("mascons/"+name+".pk", "rb") as file:
    mascon_points, mascon_masses, mascon_name = pk.load(file)
mascon_points = torch.tensor(mascon_points)
mascon_masses = torch.tensor(mascon_masses)
mascon_cube_points = torch.tensor(mascon_cube_points)
mascon_cube_masses = torch.tensor(mascon_cube_masses)

# 3 - We load the geodesyNET
encoding = gravann.direct_encoding()
model = gravann.init_network(encoding, n_neurons=100, model_type="siren", activation = gravann.AbsLayer())
model.load_state_dict(torch.load("models/"+name+".mdl"))

<All keys matched successfully>

In [5]:
results_mascon = gravann.validation_mascon(mascon_cube_points, mascon_cube_masses,mascon_points,mascon_masses, asteroid_pk_path="3dmeshes/Eros.pk", N=10000, batch_size=100, progressbar=True)

Computing validation...:  59%|█████████████████████████████                    | 44100/74244 [00:14<00:09, 3050.65it/s]

Discarding 4193 of 14744 points in altitude sampler which did not meet requested altitude.


Computing validation...:  73%|███████████████████████████████████▋             | 54100/74244 [00:29<00:06, 3092.07it/s]

Discarding 7421 of 14744 points in altitude sampler which did not meet requested altitude.


Computing validation...:  86%|██████████████████████████████████████████▎      | 64100/74244 [00:40<00:03, 3131.46it/s]

Discarding 11243 of 14744 points in altitude sampler which did not meet requested altitude.


Computing validation...: 74400it [00:48, 1537.50it/s]                                                                  


In [7]:
results_geodesyNet = gravann.validation(model, encoding, mascon_points, mascon_masses, use_acc=True, asteroid_pk_path="3dmeshes/Eros.pk", N=10000, N_integration=300000, batch_size=100, progressbar=True)

C:\ProgramData\Miniconda3\envs\geodesynet\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Computing validation...:  60%|█████████████████████████████▊                    | 44300/74244 [00:45<00:30, 972.43it/s]

Discarding 4193 of 14744 points in altitude sampler which did not meet requested altitude.


Computing validation...:  73%|████████████████████████████████████▋             | 54400/74244 [01:21<00:53, 373.85it/s]

Discarding 7421 of 14744 points in altitude sampler which did not meet requested altitude.


Computing validation...:  87%|███████████████████████████████████████████▎      | 64400/74244 [01:53<00:23, 412.06it/s]

Discarding 11243 of 14744 points in altitude sampler which did not meet requested altitude.


Computing validation...: 74400it [02:23, 516.74it/s]                                                                   


In [52]:
results_geodesyNet.to_csv("models/"+name+"_validation_results.csv")

In [8]:
results_geodesyNet

,Altitude,Normalized L1 Loss,Normalized Relative Component Loss,RMSE,relRMSE
0,Low Altitude,0.028174,0.018238,0.272122,0.055747
1,High Altitude,0.002311,0.001711,0.012548,0.002981
2,Altitude_0,0.00209,0.001599,0.007825,0.001991
3,Altitude_1,0.000416,0.000386,0.001433,0.000446
4,Altitude_2,0.000052,0.000081,0.000192,0.000099


In [6]:
results_mascon

,Altitude,Normalized L1 Loss,Normalized Relative Component Loss,RMSE,relRMSE
0,Low Altitude,0.037246,0.023903,0.347403,0.071892
1,High Altitude,0.001791,0.001315,0.010938,0.002544
2,Altitude_0,0.001436,0.001108,0.005659,0.001589
3,Altitude_1,0.000333,0.000307,0.001155,0.000352
4,Altitude_2,0.000056,0.000086,0.000317,0.000174
